# Lab 5 - Parcel Feature Extraction

Next, we will illustrate the construction of features related to our main task: finding the relationship between property development and water quality over time.  In a previous lab, you identified lakes for which we have complete information for the years from 2004 to 2015.  In this lab, we will

[Original Data and variable information](https://gisdata.mn.gov/organization/us-mn-state-metrogis?q=Metro+Regional+Parcel+Dataset&sort=score+desc%2C+metadata_modified+desc)

## Problem 1 - Feature construction

**Overview.** Remember that our target output file will have one row per year-lake combination.  To attach property information, we will need to group and aggregate the parcel data to create features for each lake-year combination.  When grouping the data, be sure to maintain the variables needed to join to the water quality data, namely the lake ID and year.  Since we are looking at tracking property development/change over time, we will want to generate features tracking

* Number of properties close to each lake,
* Summaries of the value of properties close to each lake,
* Aggregations on the size and type of the properties, and
* Other features that might impact water quality.
    
#### Task 1. Understanding parcel variables

Before we can construct features, we need to make sure we understand the parcel data.  The metro parcel data is provided by the State of Minnesota and the meta data can be found online.  For example, searching for *metro parcel 2014* lead to [this site](https://geo.btaa.org/catalog/304cf3d8-a53b-4ea9-b02a-f550bd68e320).  Clicking on the *Meta data* button in the top left, brought up more information.  Clicking *Download* opened in this meta data [in a separate page](https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_parcels_2014/metadata/metadata.html)

Look through the **Section 4: Attributes** and identify variables that might impact the water quality of near-by lakes.

> <font color="orange"> Several attributes from the Attributes section might impact the water quality. Size variables like ACRES_POLY, ACRES_DEED and development variables like FIN_SQ_FT, GARAGESQFT, and DWELL_TYPE affect the amount of surface area and potential stormwater flow entering nearby lakes. Also, TOTAL_TAX and EMV_TOTAL are value variables and may affect the water quality based on certain features about the property.  Lastly, other categoricla variables such as GARAGE, BASEMENT, and COOLING will help us identify these certain features in the value variables and may show up as very useful in the model. </font>

### Task 2. Feature Brainstorming

Our objective is to build a feature table with one row per lake-year, using grouped summary statistics. Here are effective strategies for feature construction:

1. **Numerical summaries:** Calculate group-level statistics (mean, median, standard deviation, IQR, etc.) for numeric variables.
2. **Categorical summaries:** For text data, consider:
   - **Success rates:** Compute proportions for binary variables (e.g., percent of homes with basements).
   - **Label cleaning:** Review and standardize unique labels to remove duplicates or inconsistencies.
   - **Broader categories:** Recode variables with many rare categories into a smaller, more meaningful set.
   - **Indicator columns:** Create indicator variables and aggregate them to show presence/absence or proportions (e.g., count of each property use type).

Review the variables you identified earlier and outline a feature construction strategy for each.

> <font color="orange"> Feature Construction Strategy for Lake-Year Table
1. Numerical Variables (ACRES_POLY, FIN_SQ_FT, TOTAL_TAX, EMV_TOTAL)
- Compute summary statistics for each lake/year combination

2. Binary Categorical Summaries (GARAGE, BASEMENT)
- Identify and fill null values
- Recode as binary (0,1)
- Compute % Yes for each lake/year combination

3. Other Categorical Variables (COOLING, DWELL_TYPE)
- Review and standardize unique labels
- Create a literal column
- Pivot to get counts of each type
- Create a total column and get proportions for each lake/year combination

</font>

### Task 4. Initial querying with filter and select

First, you should build a query that filters the parcel data to 
1. only include parcels within 1600 feet of the lakes we are studying, and 
2. only for the lakes with complete information.  

You should also select only the columns you will need for feature construction and joining to the water quality data.

In [1]:
import polars as pl
import polars.selectors as cs
from glob import glob
import re
from humanize import naturalsize
import os
from operator import mul

In [2]:
parcel_data = pl.scan_parquet("./data/parcel_combined.parquet")

In [3]:
# FROM LAB 4
ids_to_keep = ['82015300-01',
 '27071100-01',
 '82033400-01',
 '82011301-01',
 '82008700-01',
 '82009400-01',
 '10012100-01',
 '82010400-01',
 '10000200-01',
 '27004201-01',
 '82009002-01',
 '70002600-01',
 '82013700-01',
 '19002300-01',
 '82008900-01',
 '19002200-01',
 '82015900-01',
 '82010300-01',
 '82036800-01',
 '82011602-01',
 '82009200-01',
 '82005400-01',
 '19002700-01',
 '10001900-01',
 '19002900-01',
 '27005300-01',
 '82012300-01',
 '13005300-01',
 '27007000-01',
 '82007700-01',
 '19002400-01',
 '82009700-01',
 '19002100-01',
 '19044600-01',
 '27062700-01',
 '02000500-01',
 '82010100-01',
 '10005200-01',
 '10001100-01',
 '19003300-01',
 '19002601-01',
 '19034800-01',
 '82003400-01',
 '82012200-01',
 '82002000-01',
 '19003100-01',
 '19002500-01',
 '10009500-01',
 '27003501-01']

In [4]:
(parcel_filtered := parcel_data
    .filter(pl.col("Monit_MAP_CODE1").is_in(ids_to_keep))
    .filter(pl.col('distance_category') != 'over_1600m')
).collect()

ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,COOLING,COUNTY_ID,DWELL_TYPE,EMV_BLDG,EMV_LAND,EMV_TOTAL,FIN_SQ_FT,GARAGE,GARAGESQFT,GREEN_ACRE,HEATING,HOMESTEAD,HOME_STYLE,LANDMARK,LOT,MULTI_USES,NUM_UNITS,OPEN_SPACE,OWNER_MORE,OWNER_NAME,OWN_ADD_L1,OWN_ADD_L2,OWN_ADD_L3,PARC_CODE,PIN,PLAT_NAME,PREFIXTYPE,PREFIX_DIR,SALE_DATE,SALE_VALUE,SCHOOL_DST,SPEC_ASSES,STREETNAME,STREETTYPE,SUFFIX_DIR,Shape_Area,Shape_Leng,TAX_ADD_L1,TAX_ADD_L2,TAX_ADD_L3,TAX_CAPAC,TAX_EXEMPT,TAX_NAME,TOTAL_TAX,UNIT_INFO,USE1_DESC,USE2_DESC,USE3_DESC,USE4_DESC,WSHD_DIST,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long,Monit_MAP_CODE1,Distance_Parcel_Lake_meters,distance_category
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str
"""0.0""","""0.55""",null,null,"""N""",null,null,null,"""LINO LAKES""",null,null,"""003""",null,"""0.0""","""0.0""","""0.0""","""0.0""",null,null,"""N""",null,"""N""",null,null,null,null,null,"""N""",null,null,null,null,""",""","""0.0""","""003-043122330013""",null,null,null,null,"""0.0""","""831""","""0.0""",null,null,null,"""2236.27425472""","""404.228421402""",null,null,""",""","""0.0""","""N""",null,"""0.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""0.0""","""2004""",null,null,"""45.19766""","""-93.09914""","""02000500-01""",1439.588328,"""between_501_1600m"""
"""0.0""","""0.55""",null,null,"""N""",null,null,null,"""LINO LAKES""",null,null,"""003""",null,"""0.0""","""0.0""","""0.0""","""0.0""",null,null,"""N""",null,"""N""",null,null,null,null,null,"""N""",null,null,null,null,""",""","""0.0""","""003-043122330013""",null,null,null,null,"""0.0""","""831""","""0.0""",null,null,null,null,null,null,null,""",""","""0.0""","""N""",null,"""0.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""0.0""","""2004""",null,null,"""45.19766""","""-93.09914""","""02000500-01""",1439.588328,"""between_501_1600m"""
"""0.0""","""0.23""",null,null,"""N""",null,"""720""","""2""","""LINO LAKES""","""CIRCLE PINES""",null,"""003""",null,"""139136.0""","""64260.0""","""219654.0""","""0.0""",null,null,"""N""",null,"""Y""",null,null,"""1""",null,"""000000""","""N""",null,null,"""720 79TH ST""","""LINO LAKES""","""MN, 55014""","""0.0""","""003-083122140043""",null,null,null,"""2000-03-28""","""169900.0""","""831""","""0.0""","""79TH""","""ST""",null,"""950.166070049""","""127.290898214""","""605 STATE ST PO BOX 1868""","""LA CROSSE""","""WI, 54602""","""2099.0""","""N""",null,"""2351.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""1993.0""","""2004""",null,null,"""45.19216""","""-93.1044""","""02000500-01""",1244.325754,"""between_501_1600m"""
"""0.0""","""0.23""",null,null,"""N""",null,"""720""","""2""","""LINO LAKES""","""CIRCLE PINES""",null,"""003""",null,"""139136.0""","""64260.0""","""219654.0""","""0.0""",null,null,"""N""",null,"""Y""",null,null,"""1""",null,"""000000""","""N""",null,null,"""720 79TH ST""","""LINO LAKES""","""MN, 55014""","""0.0""","""003-083122140043""",null,null,null,"""2000-03-28""","""169900.0""","""831""","""0.0""","""79TH""","""ST""",null,null,null,"""605 STATE ST PO BOX 1868""","""LA CROSSE""","""WI, 54602""","""2099.0""","""N""",null,"""2351.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""1993.0""","""2004""",null,null,"""45.19216""","""-93.1044""","""02000500-01""",1244.325754,"""between_501_1600m"""
"""0.0""","""0.98""",null,null,"""N""",null,"""8060""",null,"""LINO LAKES""","""CIRCLE PINES""",null,"""003""",null,"""804.0""","""120650.0""","""121507.0""","""0.0""",null,null,"""N""",null,"""Y""",null,null,null,null,null,"""N""",null,n

## Problem 2.  Numerical Summaries

Two important categories of property data involve the size (e.g., finished square footage) and value (e.g., accessed value and/or taxes paid).

**Tasks.** 

1. Identify 2-3 variables for each of these categories.
2. Write a query that computes the summary statistics for each of these variables for each lake-year.  
3. Write this summary table out to a CSV file named `parcel_numerical_summaries.csv`.  Again, you should partition by lake ID and year.

2-3 variables for each of these categories:

1. Size
- ACRES_POLY
- FIN_SQ_FT

2. Value
- EMV_TOTAL
- TOTAL_TAX

In [27]:
(numerical_summary := parcel_filtered
 .select(pl.col(["ACRES_POLY", "FIN_SQ_FT", "EMV_TOTAL", "TOTAL_TAX"]).cast(pl.Float64), 'Monit_MAP_CODE1', 'Year')
 .group_by('Monit_MAP_CODE1', 'Year')
 .agg([pl.col("ACRES_POLY").mean().alias("ACRES_POLY_mean"),
       pl.col("FIN_SQ_FT").mean().alias("FIN_SQ_FT_mean"),
       pl.col("EMV_TOTAL").mean().alias("EMV_TOTAL_mean"),
       pl.col("TOTAL_TAX").mean().alias("TOTAL_TAX_mean")])
 .filter(pl.col('Year') != "2015")
 .collect()
)

Monit_MAP_CODE1,Year,ACRES_POLY_mean,FIN_SQ_FT_mean,EMV_TOTAL_mean,TOTAL_TAX_mean
str,str,f64,f64,f64,f64
"""19034800-01""","""2011""",0.256326,1910.71519,190641.960315,2358.287718
"""82008900-01""","""2012""",0.901842,1431.351641,338015.338245,0.0
"""10001100-01""","""2005""",0.695463,1873.932886,349537.449664,3803.167785
"""82009700-01""","""2004""",0.7189,3.537009,243402.040816,2024.29668
"""70002600-01""","""2014""",0.597399,1176.143634,323759.628455,0.0
…,…,…,…,…,…
"""10000200-01""","""2012""",0.934115,549.859473,508419.5734,7559.442911
"""27071100-01""","""2013""",3.699208,928.404898,334760.023701,8832.190993
"""10000200-01""","""2008""",0.929409,263.179874,561044.528302,6995.290566


In [28]:
numerical_summary.write_csv("./data/parcel_numerical_summaries.csv")

## Problem 3.  Simple categorical summaries.

In this part, you will create summary statistics for some of the simpler categorical variables.

**Binary variables.** There are two examples of binary variables, listed below.  You will need to compute the percent of `Yes` for each.

* GARAGE: Garage Y/N
* BASEMENT: Basement Y/N

**Other categorical variables.** There are a number of other categorical variables.  You need to select one of these variables, inspect/clean your variable as needed, create indicator variables for each resulting label, and compute summary statistics for each label.

* HOMESTEAD: Homestead Status
* TAX_EXEMPT: Tax Exempt Status 
* DWELL_TYPE: Dwelling Type 
* HOME_STYLE: Home Style
* HEATING: Heating type
* COOLING: Cooling type

**Tasks.**
Create a query that

1. Select one binary and two other categorical variables for feature construction,
2. Reads in the parcel data and selects the relevant columns (be sure to keep the lake ID and year),
3. Inspect unique labels and recode/clean as needed,
4. Create a literal column of ones, and
5. Pivot to get the counts of each label per lake-year (do this once per category).

Write this summary table out to a csv file named `parcel_categorical_summaries.csv`.  Again, you should partition by lake ID and year.

In [30]:
(parcel_categorical := parcel_filtered
  .select(pl.col("GARAGE").fill_null("N"), pl.col("BASEMENT").fill_null("N"), 'Monit_MAP_CODE1', 'Year')
  .with_columns(pl.when(pl.col("GARAGE") == "Y").then(1).otherwise(0).alias("GARAGE"),
                pl.when(pl.col("BASEMENT") == "Y").then(1).otherwise(0).alias("BASEMENT"))
  .group_by('Monit_MAP_CODE1', 'Year')
  .agg([pl.col("GARAGE").mean().alias("GARAGE_mean"),
        pl.col("BASEMENT").mean().alias("BASEMENT_mean")])
  .filter(pl.col('Year') != "2015")
  .collect()
 )

Monit_MAP_CODE1,Year,GARAGE_mean,BASEMENT_mean
str,str,f64,f64
"""27005300-01""","""2013""",0.721926,0.495418
"""70002600-01""","""2008""",0.0,0.0
"""82010100-01""","""2008""",0.0,0.0
"""82008700-01""","""2013""",0.8125,0.821759
"""27005300-01""","""2011""",0.695456,0.008486
…,…,…,…
"""82009400-01""","""2005""",0.0,0.0
"""19002601-01""","""2014""",0.0,0.0
"""19002200-01""","""2010""",0.0,0.0


In [7]:
(cooling_counts := parcel_filtered
 .select(pl.col("COOLING"))
 .with_columns(pl.col("COOLING").str.strip_chars().str.to_uppercase().alias("COOLING"))
 .group_by("COOLING")
 .agg(pl.len().alias("counts"))
 .sort("counts")
 .collect()
)

COOLING,counts
str,u32
"""SPACE HEATER""",2
"""GRAVITY/WA""",6
"""OTHER W A/""",8
"""STEAM W A/""",8
"""4 AC UNITS""",8
…,…
"""N""",22433
"""1 AC UNIT""",37517
"""Y""",191818


In [ ]:
(rare_categories := cooling_counts.filter(pl.col("counts") < 2000).select("COOLING").to_series().to_list())

['SPACE HEATER',
 'GRAVITY/WA',
 'OTHER W A/',
 'STEAM W A/',
 '4 AC UNITS',
 'CHILL WATR',
 'CEN.REFRIG',
 'EVAPORATIVE COOLING',
 'CEN. REFRI',
 'CEN.EVAP',
 'RAD/BBELEC',
 '3 AC UNITS',
 'WALL MOUNT',
 'CNTRL',
 'NONE',
 '2 AC UNITS',
 'CENTW/AIR',
 'HOT WATER',
 'PKG RF TOP',
 'CENTRAL']

In [33]:
(parcel_cleaned := parcel_filtered
    .with_columns(pl.col("COOLING").fill_null("UNKNOWN"))
    .with_columns(pl.col("COOLING").str.strip_chars().str.to_uppercase().alias("COOLING"))
    .with_columns(pl.when(pl.col("COOLING").is_in(rare_categories)).then(pl.lit("OTHER"))
                  .otherwise(pl.col("COOLING")).alias("COOLING"))
    .with_columns(pl.lit(1).alias("count"))
).collect()

ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,COOLING,COUNTY_ID,DWELL_TYPE,EMV_BLDG,EMV_LAND,EMV_TOTAL,FIN_SQ_FT,GARAGE,GARAGESQFT,GREEN_ACRE,HEATING,HOMESTEAD,HOME_STYLE,LANDMARK,LOT,MULTI_USES,NUM_UNITS,OPEN_SPACE,OWNER_MORE,OWNER_NAME,OWN_ADD_L1,OWN_ADD_L2,OWN_ADD_L3,PARC_CODE,PIN,PLAT_NAME,PREFIXTYPE,PREFIX_DIR,SALE_DATE,SALE_VALUE,SCHOOL_DST,SPEC_ASSES,STREETNAME,STREETTYPE,SUFFIX_DIR,Shape_Area,Shape_Leng,TAX_ADD_L1,TAX_ADD_L2,TAX_ADD_L3,TAX_CAPAC,TAX_EXEMPT,TAX_NAME,TOTAL_TAX,UNIT_INFO,USE1_DESC,USE2_DESC,USE3_DESC,USE4_DESC,WSHD_DIST,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long,Monit_MAP_CODE1,Distance_Parcel_Lake_meters,distance_category,count
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,i32
"""0.0""","""0.55""",null,null,"""N""",null,null,null,"""LINO LAKES""",null,"""UNKNOWN""","""003""",null,"""0.0""","""0.0""","""0.0""","""0.0""",null,null,"""N""",null,"""N""",null,null,null,null,null,"""N""",null,null,null,null,""",""","""0.0""","""003-043122330013""",null,null,null,null,"""0.0""","""831""","""0.0""",null,null,null,"""2236.27425472""","""404.228421402""",null,null,""",""","""0.0""","""N""",null,"""0.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""0.0""","""2004""",null,null,"""45.19766""","""-93.09914""","""02000500-01""",1439.588328,"""between_501_1600m""",1
"""0.0""","""0.55""",null,null,"""N""",null,null,null,"""LINO LAKES""",null,"""UNKNOWN""","""003""",null,"""0.0""","""0.0""","""0.0""","""0.0""",null,null,"""N""",null,"""N""",null,null,null,null,null,"""N""",null,null,null,null,""",""","""0.0""","""003-043122330013""",null,null,null,null,"""0.0""","""831""","""0.0""",null,null,null,null,null,null,null,""",""","""0.0""","""N""",null,"""0.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""0.0""","""2004""",null,null,"""45.19766""","""-93.09914""","""02000500-01""",1439.588328,"""between_501_1600m""",1
"""0.0""","""0.23""",null,null,"""N""",null,"""720""","""2""","""LINO LAKES""","""CIRCLE PINES""","""UNKNOWN""","""003""",null,"""139136.0""","""64260.0""","""219654.0""","""0.0""",null,null,"""N""",null,"""Y""",null,null,"""1""",null,"""000000""","""N""",null,null,"""720 79TH ST""","""LINO LAKES""","""MN, 55014""","""0.0""","""003-083122140043""",null,null,null,"""2000-03-28""","""169900.0""","""831""","""0.0""","""79TH""","""ST""",null,"""950.166070049""","""127.290898214""","""605 STATE ST PO BOX 1868""","""LA CROSSE""","""WI, 54602""","""2099.0""","""N""",null,"""2351.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""1993.0""","""2004""",null,null,"""45.19216""","""-93.1044""","""02000500-01""",1244.325754,"""between_501_1600m""",1
"""0.0""","""0.23""",null,null,"""N""",null,"""720""","""2""","""LINO LAKES""","""CIRCLE PINES""","""UNKNOWN""","""003""",null,"""139136.0""","""64260.0""","""219654.0""","""0.0""",null,null,"""N""",null,"""Y""",null,null,"""1""",null,"""000000""","""N""",null,null,"""720 79TH ST""","""LINO LAKES""","""MN, 55014""","""0.0""","""003-083122140043""",null,null,null,"""2000-03-28""","""169900.0""","""831""","""0.0""","""79TH""","""ST""",null,null,null,"""605 STATE ST PO BOX 1868""","""LA CROSSE""","""WI, 54602""","""2099.0""","""N""",null,"""2351.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""1993.0""","""2004""",null,null,"""45.19216""","""-93.1044""","""02000500-01""",1244.325754,"""between_501_1600m""",1
"""0.0""","""0.98""",null,null,"""N""",null,"""8060""",null,"""LINO LAKES""","""CIRCLE PINES""","""UNKNOWN""","""003""",null,"""804.0""","""120650.0""","""121507.0""","""0.0""",null,nu

In [34]:
(parcel_cooling_counts := parcel_cleaned
    .group_by(["Monit_MAP_CODE1", "Year", "COOLING"])
    .agg(pl.col("count").sum().alias("count"))
    .collect()
)

Monit_MAP_CODE1,Year,COOLING,count
str,str,str,i32
"""82013700-01""","""2009""","""OTHER""",34
"""19002400-01""","""2011""","""UNKNOWN""",2509
"""19044600-01""","""2012""","""FORCED AIR""",4457
"""82007700-01""","""2012""","""UNKNOWN""",42
"""82015300-01""","""2011""","""N""",92
…,…,…,…
"""82012300-01""","""2008""","""UNKNOWN""",343
"""27003501-01""","""2015""","""1""",2160
"""19003100-01""","""2014""","""FORCED AIR""",1095


In [35]:
(parcel_cooling_long := parcel_cooling_counts
   .pivot(values="count", index=["Monit_MAP_CODE1", "Year"], on="COOLING")
   .rename({col: f"{col}_COOLING" for col in parcel_cooling_counts["COOLING"].unique()})
   .fill_null(0)
   .with_columns(pl.sum_horizontal(cs.integer()).alias("total"))
   .with_columns((cs.integer()) / pl.col("total"))
   .drop('total')
   .filter(pl.col("Year") != "2015")
)


Monit_MAP_CODE1,Year,OTHER_COOLING,UNKNOWN_COOLING,FORCED AIR_COOLING,N_COOLING,Y_COOLING,1 AC UNIT_COOLING,A/CON_COOLING,CENTRAL W/AIR COND_COOLING,1_COOLING,0_COOLING
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""82013700-01""","""2009""",0.034068,0.519038,0.0,0.0,0.0,0.0,0.0,0.446894,0.0,0.0
"""19002400-01""","""2011""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""19044600-01""","""2012""",0.003002,0.041381,0.955617,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""82007700-01""","""2012""",0.0,0.214286,0.0,0.204082,0.581633,0.0,0.0,0.0,0.0,0.0
"""82015300-01""","""2011""",0.0,0.191824,0.0,0.144654,0.663522,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…
"""19002400-01""","""2005""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""82010300-01""","""2009""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""10000200-01""","""2004""",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
(dwelling_counts := parcel_filtered
 .select(pl.col("DWELL_TYPE"))
 .with_columns(pl.col("DWELL_TYPE").str.strip_chars().str.to_uppercase().alias("DWELL_TYPE"))
 .group_by("DWELL_TYPE")
 .agg(pl.len().alias("counts"))
 .sort("counts")
 .collect()
)

DWELL_TYPE,counts
str,u32
"""GOVERNMENTAL BUILDING""",2
"""INDUSTRIALS, LIGHT MFTG.""",2
"""1 1/4 STORY FINISHED""",2
"""RELOCATABLE CLASSROOM""",2
"""SEASONAL RECREATIONAL""",2
…,…
"""CONDOMINIUM""",17912
"""TOWNHOUSE""",130713
"""SINGLE-FAMILY / OWNER OCCUPIED""",152842


In [18]:
(rare_categories_dwell := dwelling_counts.filter(pl.col("counts") < 15000).select("DWELL_TYPE").to_series().to_list())

['GOVERNMENTAL BUILDING',
 'INDUSTRIALS, LIGHT MFTG.',
 '1 1/4 STORY FINISHED',
 'RELOCATABLE CLASSROOM',
 'SEASONAL RECREATIONAL',
 'FIRE STATION (VOLUNTEER)',
 'BAR/TAVERN',
 'RESTROOM BUILDING',
 'DISCOUNT STORE',
 'INDUSTRIAL, RESIDENTIAL SINGL*',
 'OTHER RESIDENTIAL',
 'TWO-FAMILY FLAT',
 'NEIGHBORHOOD SHOPPING CTR',
 'RETAIL STORE',
 'OFFICE BUILDING',
 'TENNS,INDR',
 'RCQT BL CB',
 'LUMBER STG',
 'EXEMPT - MUNICIPAL LAW ENFORC*',
 'SHED,UTIL',
 'DOUBLE BUNGALOW/DUPLEX',
 'STORAGE WAREHOUSE',
 'MOBILE HOME HOUSING',
 'EXEMPT - CHURCH PROPERTY',
 'TWO FAMILY DWELLING',
 'AGRICULTURAL, AGRICULTURAL,',
 'SHPCTR,RGL',
 'TAX FORFEIT',
 'BWLNGALLEY',
 'COUNTRYCLB',
 'TAX FORFEIT - DNR PAYMENT IN *',
 'PRKNG STRC',
 'TWO-FAMILY DUPLEX',
 'COMMERCIAL / GOLF COURSE, RES*',
 'HOTEL',
 'SHED,EQUIP',
 'MORTUARY',
 'SERVC STN',
 'SRV MINLBE',
 'HEALTH CLB',
 'THEATER',
 'NONE',
 'MARKET',
 'LAUNDROMAT',
 'CHURCH',
 'APARTMENTS / COOP',
 'INDL,R&D',
 'WHSE,MINI',
 'AUTO SHWRM',
 'AUTO CENTR',


In [22]:
(parcel_cleaned_dwell := parcel_filtered
    .with_columns(pl.col("DWELL_TYPE").fill_null("UNKNOWN_DWELL_TYPE"))
    .with_columns(pl.col("DWELL_TYPE").str.strip_chars().str.to_uppercase().alias("DWELL_TYPE"))
    .with_columns(pl.when(pl.col("DWELL_TYPE").is_in(rare_categories_dwell)).then(pl.lit("OTHER_DWELL"))
                  .otherwise(pl.col("DWELL_TYPE")).alias("DWELL_TYPE"))
    .with_columns(pl.lit(1).alias("count"))
).collect()

ACRES_DEED,ACRES_POLY,AGPRE_ENRD,AGPRE_EXPD,AG_PRESERV,BASEMENT,BLDG_NUM,BLOCK,CITY,CITY_USPS,COOLING,COUNTY_ID,DWELL_TYPE,EMV_BLDG,EMV_LAND,EMV_TOTAL,FIN_SQ_FT,GARAGE,GARAGESQFT,GREEN_ACRE,HEATING,HOMESTEAD,HOME_STYLE,LANDMARK,LOT,MULTI_USES,NUM_UNITS,OPEN_SPACE,OWNER_MORE,OWNER_NAME,OWN_ADD_L1,OWN_ADD_L2,OWN_ADD_L3,PARC_CODE,PIN,PLAT_NAME,PREFIXTYPE,PREFIX_DIR,SALE_DATE,SALE_VALUE,SCHOOL_DST,SPEC_ASSES,STREETNAME,STREETTYPE,SUFFIX_DIR,Shape_Area,Shape_Leng,TAX_ADD_L1,TAX_ADD_L2,TAX_ADD_L3,TAX_CAPAC,TAX_EXEMPT,TAX_NAME,TOTAL_TAX,UNIT_INFO,USE1_DESC,USE2_DESC,USE3_DESC,USE4_DESC,WSHD_DIST,XUSE1_DESC,XUSE2_DESC,XUSE3_DESC,XUSE4_DESC,YEAR_BUILT,Year,ZIP,ZIP4,centroid_lat,centroid_long,Monit_MAP_CODE1,Distance_Parcel_Lake_meters,distance_category,count
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,i32
"""0.0""","""0.55""",null,null,"""N""",null,null,null,"""LINO LAKES""",null,null,"""003""","""UNKNOWN_DWELL_TYPE""","""0.0""","""0.0""","""0.0""","""0.0""",null,null,"""N""",null,"""N""",null,null,null,null,null,"""N""",null,null,null,null,""",""","""0.0""","""003-043122330013""",null,null,null,null,"""0.0""","""831""","""0.0""",null,null,null,"""2236.27425472""","""404.228421402""",null,null,""",""","""0.0""","""N""",null,"""0.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""0.0""","""2004""",null,null,"""45.19766""","""-93.09914""","""02000500-01""",1439.588328,"""between_501_1600m""",1
"""0.0""","""0.55""",null,null,"""N""",null,null,null,"""LINO LAKES""",null,null,"""003""","""UNKNOWN_DWELL_TYPE""","""0.0""","""0.0""","""0.0""","""0.0""",null,null,"""N""",null,"""N""",null,null,null,null,null,"""N""",null,null,null,null,""",""","""0.0""","""003-043122330013""",null,null,null,null,"""0.0""","""831""","""0.0""",null,null,null,null,null,null,null,""",""","""0.0""","""N""",null,"""0.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""0.0""","""2004""",null,null,"""45.19766""","""-93.09914""","""02000500-01""",1439.588328,"""between_501_1600m""",1
"""0.0""","""0.23""",null,null,"""N""",null,"""720""","""2""","""LINO LAKES""","""CIRCLE PINES""",null,"""003""","""UNKNOWN_DWELL_TYPE""","""139136.0""","""64260.0""","""219654.0""","""0.0""",null,null,"""N""",null,"""Y""",null,null,"""1""",null,"""000000""","""N""",null,null,"""720 79TH ST""","""LINO LAKES""","""MN, 55014""","""0.0""","""003-083122140043""",null,null,null,"""2000-03-28""","""169900.0""","""831""","""0.0""","""79TH""","""ST""",null,"""950.166070049""","""127.290898214""","""605 STATE ST PO BOX 1868""","""LA CROSSE""","""WI, 54602""","""2099.0""","""N""",null,"""2351.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""1993.0""","""2004""",null,null,"""45.19216""","""-93.1044""","""02000500-01""",1244.325754,"""between_501_1600m""",1
"""0.0""","""0.23""",null,null,"""N""",null,"""720""","""2""","""LINO LAKES""","""CIRCLE PINES""",null,"""003""","""UNKNOWN_DWELL_TYPE""","""139136.0""","""64260.0""","""219654.0""","""0.0""",null,null,"""N""",null,"""Y""",null,null,"""1""",null,"""000000""","""N""",null,null,"""720 79TH ST""","""LINO LAKES""","""MN, 55014""","""0.0""","""003-083122140043""",null,null,null,"""2000-03-28""","""169900.0""","""831""","""0.0""","""79TH""","""ST""",null,null,null,"""605 STATE ST PO BOX 1868""","""LA CROSSE""","""WI, 54602""","""2099.0""","""N""",null,"""2351.0""",null,null,null,null,null,"""RICE CREEK WATERSHED DISTRICT""",null,null,null,null,"""1993.0""","""2004""",null,null,"""45.19216""","""-93.1044""","""02000500-01""",1244.325754,"""between_501_1600m""",1
"""0.0""","""0.98""",null,null,"""N""",null,"""8060""",null,"""LINO LAKES""","""CIRCLE PINES""",null,"""003""","""UNKNOWN_DWELL_TYPE""","""8

In [23]:
(parcel_dwell_counts := parcel_cleaned_dwell
    .group_by(["Monit_MAP_CODE1", "Year", "DWELL_TYPE"])
    .agg(pl.col("count").sum().alias("count"))
    .collect()
)

Monit_MAP_CODE1,Year,DWELL_TYPE,count
str,str,str,i32
"""82008900-01""","""2012""","""CONDOMINIUM""",432
"""82008900-01""","""2014""","""UNKNOWN_DWELL_TYPE""",532
"""82009002-01""","""2012""","""OTHER_DWELL""",4
"""19034800-01""","""2010""","""OTHER_DWELL""",552
"""27004201-01""","""2009""","""UNKNOWN_DWELL_TYPE""",6879
…,…,…,…
"""19002700-01""","""2014""","""S.FAM.RES""",1390
"""19044600-01""","""2006""","""OTHER_DWELL""",36
"""19002601-01""","""2005""","""S.FAM.RES""",3681


In [24]:
(parcel_dwell_long := parcel_dwell_counts
   .pivot(values="count", index=["Monit_MAP_CODE1", "Year"], on="DWELL_TYPE")
   .fill_null(0)
   .with_columns(pl.sum_horizontal(cs.integer()).alias("total"))
   .with_columns((cs.integer()) / pl.col("total"))
   .drop('total')
   .filter(pl.col("Year") != "2015")
)


Monit_MAP_CODE1,Year,CONDOMINIUM,UNKNOWN_DWELL_TYPE,OTHER_DWELL,SINGLE-FAMILY / OWNER OCCUPIED,TOWNHOUSE,S.FAM.RES
str,str,f64,f64,f64,f64,f64,f64
"""82008900-01""","""2012""",0.144675,0.147354,0.0,0.303416,0.404555,0.0
"""82008900-01""","""2014""",0.140351,0.17284,0.0,0.294347,0.392463,0.0
"""82009002-01""","""2012""",0.062865,0.055556,0.000731,0.788012,0.092836,0.0
"""19034800-01""","""2010""",0.0,0.026531,0.093878,0.0,0.361054,0.518537
"""27004201-01""","""2009""",0.0,1.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…
"""10001100-01""","""2014""",0.0,1.0,0.0,0.0,0.0,0.0
"""82008900-01""","""2007""",0.0,1.0,0.0,0.0,0.0,0.0
"""82036800-01""","""2008""",0.0,1.0,0.0,0.0,0.0,0.0


In [36]:
(joined_categorical := parcel_categorical
    .join(parcel_cooling_long, on=["Monit_MAP_CODE1", "Year"], how="inner")
    .join(parcel_dwell_long, on=["Monit_MAP_CODE1", "Year"], how="inner")
)

Monit_MAP_CODE1,Year,GARAGE_mean,BASEMENT_mean,OTHER_COOLING,UNKNOWN_COOLING,FORCED AIR_COOLING,N_COOLING,Y_COOLING,1 AC UNIT_COOLING,A/CON_COOLING,CENTRAL W/AIR COND_COOLING,1_COOLING,0_COOLING,CONDOMINIUM,UNKNOWN_DWELL_TYPE,OTHER_DWELL,SINGLE-FAMILY / OWNER OCCUPIED,TOWNHOUSE,S.FAM.RES
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""82008900-01""","""2012""",0.778969,0.852646,0.0,0.147354,0.0,0.00067,0.851976,0.0,0.0,0.0,0.0,0.0,0.144675,0.147354,0.0,0.303416,0.404555,0.0
"""82008900-01""","""2014""",0.755686,0.82716,0.0,0.17284,0.0,0.00065,0.826511,0.0,0.0,0.0,0.0,0.0,0.140351,0.17284,0.0,0.294347,0.392463,0.0
"""82009002-01""","""2012""",0.944079,0.944444,0.0,0.055556,0.0,0.013889,0.930556,0.0,0.0,0.0,0.0,0.0,0.062865,0.055556,0.000731,0.788012,0.092836,0.0
"""19034800-01""","""2010""",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026531,0.093878,0.0,0.361054,0.518537
"""27004201-01""","""2009""",0.913941,0.036633,0.0,0.248728,0.751272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""10001100-01""","""2014""",0.895425,0.0,0.010458,0.129412,0.0,0.0,0.0,0.860131,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"""82008900-01""","""2007""",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"""82036800-01""","""2008""",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [37]:
joined_categorical.write_csv("./data/parcel_categorical_summaries.csv")

## Problem 4.  Join all the summaries.

Finally, you need to join all the summaries created above, along with the water quality summaries created in a previous lab, into one overall summary file.  Write the resulting table to a CSV file named `water_quality_and_parcel_summaries_2004_to_2015.csv`.

In [43]:
(water_quality_parcels :=
 pl.scan_parquet("./data/water_quality_by_year.parquet")
   .with_columns(pl.col('Year').cast(pl.String()).alias('Year'))
   .collect()
   .join(numerical_summary, left_on=["DNR_ID_Site_Number", "Year"], right_on=["Monit_MAP_CODE1", "Year"], how="inner")
   .join(joined_categorical, left_on=["DNR_ID_Site_Number", "Year"], right_on=["Monit_MAP_CODE1", "Year"], how="inner") 
 )

DNR_ID_Site_Number,Year,LAKE_NAME,avg_secchi_depth,avg_total_phosphorus,ACRES_POLY_mean,FIN_SQ_FT_mean,EMV_TOTAL_mean,TOTAL_TAX_mean,GARAGE_mean,BASEMENT_mean,OTHER_COOLING,UNKNOWN_COOLING,FORCED AIR_COOLING,N_COOLING,Y_COOLING,1 AC UNIT_COOLING,A/CON_COOLING,CENTRAL W/AIR COND_COOLING,1_COOLING,0_COOLING,CONDOMINIUM,UNKNOWN_DWELL_TYPE,OTHER_DWELL,SINGLE-FAMILY / OWNER OCCUPIED,TOWNHOUSE,S.FAM.RES
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""82008900-01""","""2012""","""Markgraf Lake""",0.359091,0.148182,0.901842,1431.351641,338015.338245,0.0,0.778969,0.852646,0.0,0.147354,0.0,0.00067,0.851976,0.0,0.0,0.0,0.0,0.0,0.144675,0.147354,0.0,0.303416,0.404555,0.0
"""82008900-01""","""2014""","""Markgraf Lake""",0.964333,0.082083,0.864665,1388.569201,376544.834308,0.0,0.755686,0.82716,0.0,0.17284,0.0,0.00065,0.826511,0.0,0.0,0.0,0.0,0.0,0.140351,0.17284,0.0,0.294347,0.392463,0.0
"""82009002-01""","""2012""","""Wilmes Lake""",1.3,0.092538,0.463816,1769.156433,250127.55848,0.0,0.944079,0.944444,0.0,0.055556,0.0,0.013889,0.930556,0.0,0.0,0.0,0.0,0.0,0.062865,0.055556,0.000731,0.788012,0.092836,0.0
"""19034800-01""","""2010""","""Valley Lake""",1.571429,0.070571,0.254844,1688.303912,191261.27551,2441.579762,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026531,0.093878,0.0,0.361054,0.518537
"""27004201-01""","""2009""","""Twin Lake""",2.127594,0.07425,0.541326,1048.443524,225295.813345,4032.902893,0.913941,0.036633,0.0,0.248728,0.751272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""10001100-01""","""2014""","""St. Joe Lake""",2.485714,0.021714,0.640889,2034.265359,388570.065359,4587.40915,0.895425,0.0,0.010458,0.129412,0.0,0.0,0.0,0.860131,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"""82008900-01""","""2007""","""Markgraf Lake""",0.4,0.157,0.0,0.0,399197.073546,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"""82036800-01""","""2008""","""Klawitter Pond""",0.596923,0.092462,3.7903,0.0,420461.350844,412444.840525,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [44]:
water_quality_parcels.write_csv('./data/water_quality_and_parcel_summaries_2004_to_2015.csv')

## Problem 5.  Put it all together

It is often useful to package all of the data constructions steps together in one convenient place.  Your last task is to

1. Gather all of your data construction code below.
    * You don't need to include exploratory code, e.g., exploring join mismatches; only the code necessary to combine, clean, and write your data.
2. Clean/refactor the code.
3. Be sure to display all important intermediate results.